<a href="https://colab.research.google.com/github/FatemeFouladkar/SVG-To-TXT/blob/main/convert_svg_to_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install svglib google-cloud-vision

In [ ]:
import os
from google.cloud import vision
import io
from google.oauth2 import service_account

# Add credentials for google cloud API
creds = service_account.Credentials.from_service_account_file('key.json')

def detect_text(input_file_path, output_file_path):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient(credentials=creds,)

    with io.open(input_file_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    # Write the result in the output file
    with open(output_file_path, "w") as output:
      output.write(texts[0].description)

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [ ]:
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
from pathlib import Path

def svg_to_txt(input):
    """Converts SVG to PNG and passes it to Google API."""
    drawing = svg2rlg(input)
    if drawing:
      png_file_name = input.split('.')[0] + ".png"
      # Convert SVG to PNG
      renderPM.drawToFile(drawing, png_file_name, fmt="PNG")
    
    # Pass to Google cloud API
    txt_file_path = Path(input.split(".")[0] + ".txt")
    detect_text(png_file_name, txt_file_path)


svg_to_txt('sample_file.svg')
